In [4]:
import pandas as pd
import psycopg2
import Constants

POSTGRES_PW = Constants.POSTGRES_PW

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="ura_data", user="postgres", password=POSTGRES_PW, host="localhost"
)

# Load the data into a Pandas DataFrame
query = "SELECT * FROM property_transactions" # 21253 Records of data
original_df = pd.read_sql(query, conn)

conn.close()

/var/folders/w8/6rq6vc9x08vdnq0683sjchs80000gn/T/ipykernel_96601/2703379235.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  original_df = pd.read_sql(query, conn)


In [5]:
df = original_df.copy()

In [6]:

df['contract_date'] = pd.to_datetime(df['contract_date'].apply(lambda x: x[:2] + '/' + x[2:]), format='%m/%y')

# Drop these records (2) as there is no lease year
df = df[df['tenure'] != '99 years leasehold']


In [7]:
# Extract lease year and contract year and months
df['lease_year'] = df['tenure'].apply(lambda x: int(x[-5:]))

df.loc[:, 'contract_year'] = df['contract_date'].dt.year
df.loc[:, 'contract_month'] = df['contract_date'].dt.month

# Drop unncessary columns (id, tenure, contract_date), and columns which only have 1 unique value (property_type, market_segment, type_of_area)
df = df.drop(['id', 'tenure','property_type','market_segment','type_of_area', 'contract_date'],axis=1)


In [8]:
# Filter data to only use data which has contract year after 5 or 10 years since the lease year. Only concern with these two specific points of time
filtered_df = df[(df['contract_year'] == df['lease_year'] + 5) | (df['contract_year'] == df['lease_year'] + 10)]

filtered_df.head()

,project_name,area,price,floor_range,type_of_sale,district,street,latitude,longitude,no_of_units,lease_year,contract_year,contract_month
15,RIVERPARC RESIDENCE,100.0,955000.0,06-10,3,19,PUNGGOL DRIVE,42209.54801,37324.10899,1,2010,2020,2
16,RIVERPARC RESIDENCE,115.0,1080000.0,06-10,3,19,PUNGGOL DRIVE,42209.54801,37324.10899,1,2010,2020,7
18,RIVERPARC RESIDENCE,100.0,970000.0,11-15,3,19,PUNGGOL DRIVE,42209.54801,37324.10899,1,2010,2020,1
19,RIVERPARC RESIDENCE,115.0,1080000.0,06-10,3,19,PUNGGOL DRIVE,42209.54801,37324.10899,1,2010,2020,1
22,RIVERPARC RESIDENCE,100.0,980000.0,11-15,3,19,PUNGGOL DRIVE,42209.54801,37324.10899,1,2010,2020,1


In [9]:
from sklearn import preprocessing

label_encoder_project_name = preprocessing.LabelEncoder()
label_encoder_street = preprocessing.LabelEncoder()
label_encoder_floor_range = preprocessing.LabelEncoder()

# Encode the categorical data
filtered_df.loc[:, 'project_name'] = label_encoder_project_name.fit_transform(filtered_df['project_name'])
filtered_df.loc[:, 'street'] = label_encoder_street.fit_transform(filtered_df['street'])
filtered_df.loc[:, 'floor_range'] = label_encoder_floor_range.fit_transform(filtered_df['floor_range'])

dtypes_dict = {"project_name": int, "street": int, "floor_range": int, 'type_of_sale':int, 'district': int}
filtered_df = filtered_df.astype(dtypes_dict)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Standardise these values as it has a large range
filtered_df.loc[:,['area', 'latitude', 'longitude']] = scaler.fit_transform(filtered_df[['area', 'latitude', 'longitude']])

In [11]:
filtered_df

,project_name,area,price,floor_range,type_of_sale,district,street,latitude,longitude,no_of_units,lease_year,contract_year,contract_month
15,22,-0.136379,955000.0,1,3,19,15,0.107802,0.843993,1,2010,2020,2
16,22,0.551220,1080000.0,1,3,19,15,0.107802,0.843993,1,2010,2020,7
18,22,-0.136379,970000.0,2,3,19,15,0.107802,0.843993,1,2010,2020,1
19,22,0.551220,1080000.0,1,3,19,15,0.107802,0.843993,1,2010,2020,1
22,22,-0.136379,980000.0,2,3,19,15,0.107802,0.843993,1,2010,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14980,18,-0.732299,1220000.0,2,3,27,20,1.801352,-0.633940,1,2014,2024,7
14981,18,-0.182219,1480000.0,2,3,27,20,1.801352,-0.633940,1,2014,2024,7
14984,18,-0.182219,1350000.0,0,3,27,20,1.801352,-0.633940,1,2014,2024,1
14988,18,-0.548939,1250000.0,1,3,27,20,1.801352,-0.633940,1,2014,2024,3


In [13]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Define features and target variable
X = filtered_df.drop(columns=['price'])
y = filtered_df['price']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize and train the model
model = RandomForestRegressor(n_estimators=200, random_state=123)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 5345521137.18324


In [16]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the models to be evaluated
models = {
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'XGBoost': XGBRegressor(eval_metric='rmse')
}

# Define the hyperparameter grids for each model
param_distributions = {
    'RandomForest': {
        'n_estimators': [200,350,500],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'GradientBoosting': {
        'n_estimators': [200,350,500],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    },
    'XGBoost': {
        'n_estimators': [200,350,500],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.5, 0.8, 1.0]
    }
}

# Initialize a dictionary to hold the results
results = {}

# Iterate through each model and perform RandomizedSearchCV
for model_name, model in models.items():
    print(f"Running RandomizedSearchCV for {model_name}...")
    
    random_search = RandomizedSearchCV(
        model,
        param_distributions[model_name],
        n_iter=50,  # Number of different combinations to try
        cv=3,       # Number of folds for cross-validation
        scoring='neg_mean_squared_error',  # Use negative MSE for scoring
        random_state=123,
        n_jobs=-1   # Use all available cores
    )
    
    random_search.fit(X_train, y_train)
    
    # Store the best model and its MSE
    best_model = random_search.best_estimator_
    best_mse = -random_search.best_score_  # Convert back to positive MSE
    
    results[model_name] = {
        'Best Parameters': random_search.best_params_,
        'Best MSE': best_mse
    }

    # Evaluate on the test set
    y_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    results[model_name]['Test MSE'] = test_mse

# Display the results
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    print(f"Best Parameters: {metrics['Best Parameters']}")
    print(f"Best Cross-Validation MSE: {metrics['Best MSE']}")
    print(f"Test MSE: {metrics['Test MSE']}")


Running RandomizedSearchCV for RandomForest...
Running RandomizedSearchCV for GradientBoosting...
Running RandomizedSearchCV for XGBoost...

RandomForest:
Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 30}
Best Cross-Validation MSE: 5052855331.926208
Test MSE: 5589740259.149956

GradientBoosting:
Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 3, 'learning_rate': 0.1}
Best Cross-Validation MSE: 3659445143.404833
Test MSE: 5178701059.759409

XGBoost:
Best Parameters: {'subsample': 0.8, 'n_estimators': 350, 'max_depth': 3, 'learning_rate': 0.1}
Best Cross-Validation MSE: 3526830600.622919
Test MSE: 5656213616.361376


In [18]:
# Best parameters
best_params = {'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 3, 'learning_rate': 0.1}

# Initialize the model with the best parameters
best_model = GradientBoostingRegressor(**best_params,random_state=37)

# Fit the model on the training data
best_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 5157003605.138978


In [20]:
import joblib

# Save the trained model
joblib.dump(best_model, 'EC_price_model.pkl')
joblib.dump(label_encoder_project_name, 'label_encoder_project_name.pkl')
joblib.dump(label_encoder_street, 'label_encoder_street.pkl')
joblib.dump(label_encoder_floor_range, 'label_encoder_floor_range.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']